In [ ]:
# Import Dependencies
# Use Flask to render a template, redirecting to another url, and creating a url
from flask import Flask, render_template, redirect, url_for
# Use PyMongo to interact with our Mongo database
from flask_pymongo import PyMongo
# Use the scraping code, we'll convert from Jupyter Notebook to Python
import scraping

In [ ]:
# Set up Flask
app = Flask(__name__)

In [ ]:
# Use flask_pymongo to set up mongo connection
# Tells Python that our app will connect to Mongo using a URI, a uniform resource identifier similar to a URL
app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
# The URI we'll use to connect our app to Mongo. Setting database name to "mars_app"
mongo = PyMongo(app)

In [ ]:
# Setting up routes
@app.route("/")
def index():
    mars = mongo.db.mars.find_one()
    return render_template("index.html", mars = mars)

In [ ]:
# Setting up the scrape route
@app.route("/scrape")
def scrape():
    # Assign a new variable that points to our Mongo database
    mars = mongo.db.mars
    # New variable to hold newly scrped data
    mars_data = scraping.scrape_all() # scrape_all fuction from the scraping.py
    # '{}' adds an empty JSON object 
    # 'mars_data' uses the data we already have stored
    # 'upsert=True' indicated to Mongo to create a new document if one doesn't already exist, and new data will be saved
    mars.update({}, mars_data, upsert=True)
    # Redirect to '/' (home page) after scraping the data
    return redirect('/', code = 302)

In [ ]:
# Run flask
if __name__ == "__main__":
    app.run()

In [ ]:
# Create a new function to scrape mars_news, '(browser)' is telling Python 
# that we'll be using the browser defined outside the function
def mars_news(browser):

    # Scrape Mars News
    # Visit the mars nasa news site
    url = 'https://redplanetscience.com/'
    browser.visit(url)

    # Optional delay for loading the page
    browser.is_element_present_by_css('div.list_text', wait_time=1)

    # Convert the browser html to a soup object and then quit the browser
    html = browser.html
    news_soup = soup(html, 'html.parser')

    # Add try/except for error handling
    try:
        slide_elem = news_soup.select_one('div.list_text')
        # Use the parent element to find the first 'a' tag and save it as 'news_title'
        news_title = slide_elem.find('div', class_='content_title').get_text()
        # Use the parent element to find the paragraph text
        news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

    except AttributeError:
        return None, None

    return news_title, news_p

In [ ]:
# Declare the function
def featured_image(browser):
    
    # Visit the url
    url = 'https://spaceimages-mars.com'
    browser.visit(url)
    
    # Find and click the full image button
    full_image_elem = browser.find_by_tag('button')[1]
    full_image_elem.click()
    
    # Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')
    
    # Find the relative image url
    try:
        # find the relative image url
        img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')

    except AttributeError:
        return None
        
        # Use the base URL to create an absolute URL
        img_url = f'https://spaceimages-mars.com/{img_url_rel}'
    
    # Return
    return img_url

In [ ]:
# Define mars_facts
def mars_facts():    # Add try/except for error handling
    
    try:
        # Use 'read_html' to scrape the facts table into a dataframe
        df = pd.read_html('https://galaxyfacts-mars.com')[0]

    except BaseException:
        return None

    # Assign columns and set index of dataframe
    df.columns=['Description', 'Mars', 'Earth']
    df.set_index('Description', inplace=True)

    # Convert dataframe into HTML format, add bootstrap
    return df.to_html()